# OBJECTIVE : To Create models for each mall

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import glob
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
# train_paths = glob.glob('../input/indoor-location-navigation/train/*/*/*')
from main_io_comp_vis_all import calibrate_magnetic_wifi_ibeacon_to_position, read_data_file
floor_txt_num = {'F1':1,'1F':1,'L1':1,'1L':1,'F2':3,'2F':3,'L2':3,'2L':3,'F3':4,'3F':4,'L3':4,'3L':4,'F4':5,'4F':5,'L4':5,'4L':5,
                'F5':6,'5F':6,'L5':6,'5L':6,'F6':7,'6F':7,'L6':7,'6L':7,'F7':8,'7F':8,'L7':8,'7L':8,'F8':9,'8F':9,'L8':9,'8L':9,
                'F9':10,'9F':10,'L9':10,'9L':10,'F10':11,'10F':11,'L10':11,'10L':11,'F11':12,'11F':12,'L11':12,'11L':12,
                'B1':1,'1B':1,"B":1,'B2':2,'2B':2,'B3':3,'3B':3,'B4':4,'4B':4}

# 1) Create Mall list & Check existence of aux data

In [ ]:
mall_max_fxy = pd.read_csv('../input/bssid-maxfxy-unique/max_fxy.csv')
train_mall_df = glob.glob('../input/mall-bssid-df/*')
train_mall_bssid_unique = glob.glob('../input/bssid-maxfxy-unique/*')
malllist=[]
for mall in train_mall_df:
    malllist.append(mall[23:-7])

i=0
for mall in malllist:
    if (('../input/bssid-maxfxy-unique/'+mall+'_bssid.csv') not in train_mall_bssid_unique) or (mall not in list(mall_max_fxy['mall'])):
        print("Data Not Present for mall: ", mall)
        print(i)
    i+=1

# 5) Fit data & Create Simple Model

In [ ]:
hist_loss =[]
mall_max_fxy = pd.read_csv('../input/bssid-maxfxy-unique/max_fxy.csv')
for idx, mall in enumerate(malllist):
    mall_df = pd.read_csv('../input/mall-bssid-df/'+mall+'_df.csv')
    mall_scaled_df = mall_df.copy()
    mall_scaled_df["floor"] /= int(mall_max_fxy[mall_max_fxy["mall"]==mall]["max_floor"])
    mall_scaled_df["x"] /= int(mall_max_fxy[mall_max_fxy["mall"]==mall]["max_x"])
    mall_scaled_df["y"] /= int(mall_max_fxy[mall_max_fxy["mall"]==mall]["max_y"])
    #print(mall_scaled_df.columns[0])
    for col in mall_scaled_df.columns[4:]:
        mall_scaled_df[col] = (mall_scaled_df[col]+100)/100
    model = Sequential()
    model.add(Dense(4000, input_dim=(len(mall_scaled_df.columns)-4), kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(120, activation='sigmoid'))
    model.add(Dense(3, activation='linear'))
    optimizer = keras.optimizers.Adam(lr=0.01)
    # model.compile(loss='mse', optimizer=optimizer)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse','mae'])
    n_epoch=5
    history = model.fit(mall_scaled_df[mall_scaled_df.columns[4:]], mall_scaled_df[mall_scaled_df.columns[1:4]], epochs=n_epoch, batch_size=16, verbose=2, validation_split=0.2)
    model.save(mall+'_mdl')
    hist_loss.append([mall]+history.history["loss"])
pd.DataFrame(data=hist_loss,columns=(["mall"]+list(range(n_epoch)))).to_csv('train_hist_mall.csv')   